In [47]:
#packages used
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import numpy

In [48]:
#importing data as pandas data frame
DevData = pd.read_json("dev.jsonl", lines = True)
TestData = pd.read_json("test.jsonl", lines = True)
TrainData = pd.read_json("train.jsonl", lines = True)

In [49]:
TrainData

,premise,choice1,choice2,question,label,idx
0,My body cast a shadow over the grass.,The sun was rising.,The grass was cut.,cause,0,0
1,The woman tolerated her friend's difficult beh...,The woman knew her friend was going through a ...,The woman felt that her friend took advantage ...,cause,0,1
2,The women met for coffee.,The cafe reopened in a new location.,They wanted to catch up with each other.,cause,1,2
3,The runner wore shorts.,The forecast predicted high temperatures.,She planned to run along the beach.,cause,0,3
4,The guests of the party hid behind the couch.,It was a surprise party.,It was a birthday party.,cause,0,4
...,...,...,...,...,...,...
345,The photograph faded.,It was fake.,It was old.,cause,1,345
346,The boy got a black eye.,The bully mocked the boy.,The bully punched the boy.,cause,1,346
347,I misplaced my wallet.,I retrieved my phone.,I retraced my steps.,effect,1,347
348,The hunter ran out of ammunition.,He reloaded the gun.,He aimed at the deer.,effect,0,348


In [50]:
#tokenizes test data
TrainTok = Tokenizer(lower=True, split=' ',)

TrainTok.fit_on_texts(TrainData['choice2'])        
TrainTok.fit_on_texts(TrainData['choice1'])
TrainTok.fit_on_texts(TrainData['premise'])


In [51]:
TrainConfig = TrainTok.get_config()

#dictionary of words and indexs
TrainVocab = eval(TrainConfig['index_word'])

#finds length of vocab
TrainVocabSize = len(TrainVocab)

print("there are ", TrainVocabSize, " unique words")

there are  1740  unique words


In [64]:
PremiseTokTrain = TrainTok.texts_to_sequences(TrainData['premise'])
Choice1TokTrain = TrainTok.texts_to_sequences(TrainData['choice1'])
Choice2TokTrain = TrainTok.texts_to_sequences(TrainData['choice2'])

PremisePadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in PremiseTokTrain) + 2)
Choice1PadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in Choice1TokTrain) + 2)
Choice2PadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in Choice2TokTrain) + 2)


[[15, 1276, 394, 2, 1277, 43, 1, 308],
 [1, 13, 1278, 6, 1279, 486, 1280],
 [1, 1281, 544, 17, 294],
 [1, 1282, 285, 1283],
 [1, 157, 14, 1, 62, 160, 116, 1, 451],
 [1, 1284, 61, 1, 1285],
 [1, 296, 57, 24, 14, 1, 63],
 [1, 16, 26, 2, 1286, 11, 9, 1287],
 [1, 183, 1288, 1, 67],
 [1, 128, 284, 2, 1289, 18, 1, 1290, 1291],
 [1, 13, 328, 6, 184],
 [1, 1292, 13, 301, 2, 1293],
 [1, 187, 244, 43, 17, 1, 474],
 [1, 1294, 1295, 1296],
 [3, 108, 92, 11, 15, 482, 25],
 [1, 19, 1297],
 [1, 63, 1298],
 [12, 26, 302, 1299],
 [3, 533, 27, 1, 300],
 [1, 64, 190, 545, 21, 10, 1, 546],
 [1, 19, 46, 2, 1300],
 [1, 64, 1301, 35, 4, 1302, 432, 2, 1303],
 [1, 1304, 303, 1305],
 [1, 123, 447, 1, 62],
 [1, 13, 68, 1, 16, 6, 300, 1306],
 [1, 1307, 523, 1308, 4, 1, 173],
 [1, 547, 39, 1309],
 [1, 101, 39, 1310, 4, 1, 192],
 [1, 1311, 133, 1312],
 [1, 1313, 226, 1314, 11, 1315],
 [1, 13, 1316, 4, 82],
 [1, 16, 1317, 9, 59, 1318],
 [1, 40, 462, 34, 1319, 38, 1, 105],
 [1, 67, 5, 1320],
 [1, 19, 175, 1, 548, 196

In [223]:
A = numpy.zeros((TrainVocabSize,TrainVocabSize))
rows = len(TrainData)
for i in range(rows):
    premiceTok = PremiseTokTrain[i]
    if TrainData["label"][i] == 1:
        sentanceTok = Choice2TokTrain[i]
    else:
        sentanceTok = Choice1TokTrain[i]
    for j in premiceTok:
        for k in sentanceTok:
            A[j-1,k-1] += 1

for i in range(len(A)):
    su = sum(A[i])
    for j in range(len(A[i])):
        A[i][j] = A[i][j]/su


/var/folders/93/zqmxc7l906ld0t2tr5sz5gjm0000gn/T/ipykernel_16626/500900994.py:16: RuntimeWarning: invalid value encountered in double_scalars
  A[i][j] = A[i][j]/su


In [243]:
PremiseTokDev = TrainTok.texts_to_sequences(DevData['premise'])
Choice1TokDev = TrainTok.texts_to_sequences(DevData['choice1'])
Choice2TokDev = TrainTok.texts_to_sequences(DevData['choice2'])

right = 0

for i in range(50):
    sum2 = 1
    sum1 = 1
    result = 0
    for j in PremiseTokDev[i]:
        for k in Choice2TokDev[i]:
            sum2 += A[j-1,k-1]
        for k in Choice1TokDev[i]:
            sum1 += A[j-1,k-1]
    if sum2/len(Choice2TokDev[i]) > sum1/len(Choice1TokDev[i]):
        result = 1
    else:
        result = 0
    right += (result == DevData["label"][i])

right/50


0.52

In [244]:
PremiseTokT = TrainTok.texts_to_sequences(TestData['premise'])
Choice1TokT = TrainTok.texts_to_sequences(TestData['choice1'])
Choice2TokT = TrainTok.texts_to_sequences(TestData['choice2'])

right = 0
for i in range(100):
    sum2 = 0
    sum1 = 0
    result = 0
    for j in PremiseTokT[i]:
        for k in Choice1TokT[i]:
            sum2 += A[j-1,k-1]
        for k in Choice2TokT[i]:
            sum1 += A[j-1,k-1]
    if sum2/len(Choice2TokT[i]) > sum1/len(Choice1TokT[i]):
        result = 1
    else:
        result = 0
    right += (result == TestData["label"][i])

right

61